SAT version for the MCP problem given in the Combinatorial Decision Making and Optimization course.

The model is based on the one already developed for the CP version of the problem with the necessary modifications to make it work for the SAT version.

Necessary libraries:

In [1]:
!pip3 install z3-solver

Necessary imports:

In [5]:
from z3 import * # The Z3 Theorem Prover
import numpy as np # Numpy for matrix operations
import matplotlib.pyplot as plt # Matplotlib for plotting

The variable instances (like number of couriers) are defined in a .dat file. The file is read and the variables are defined.

In [82]:
# open the file in Instances folder
f = open("Instances/inst01.dat", "r")
# the first line is the number of couriers
m = int(f.readline())
# the second line is the number of items
n = int(f.readline())
# the third line is the load size of each courier
load_size = [int(x) for x in f.readline().split()]
# the fourth line is the size of each item
item_size = [int(x) for x in f.readline().split()]
# the rest is the distance matrix
distance = []
for i in range(n+1):
    distance.append([int(x) for x in f.readline().split()])
# close the file
f.close()
print("couriers:", m)
print("items:", n)
print("load_size:", load_size)
print("item_size:", item_size)
# output the distance matrix as a numpy array
distance = np.array(distance)
print("distance:\n", distance)

couriers: 2
items: 6
load_size: [15, 10]
item_size: [3, 2, 6, 5, 4, 4]
distance:
 [[0 3 4 5 6 6 2]
 [3 0 1 4 5 7 3]
 [4 1 0 5 6 6 4]
 [4 4 5 0 3 3 2]
 [6 7 8 3 0 2 4]
 [6 7 8 3 2 0 4]
 [2 3 4 3 4 4 0]]


Other variables:

In [83]:
max_path_length = n - (m - 1) + 2
PATH = range(max_path_length)
COURIERS = np.arange(m)
ITEMS = np.arange(n)
items_total = sum(item_size)
row_empty = (n) * max_path_length
ITEMS_PATH = np.arange(1, max_path_length-1)
# path array 
paths = [[Int(f'paths_{c}_{p}') for p in PATH] for c in COURIERS]
paths

[[paths_0_0, paths_0_1, paths_0_2, paths_0_3, paths_0_4, paths_0_5, paths_0_6],
 [paths_1_0, paths_1_1, paths_1_2, paths_1_3, paths_1_4, paths_1_5, paths_1_6]]

In [84]:
ITEMS_PATH

array([1, 2, 3, 4, 5])

Formulation of the constraints:

In [85]:
constraints = []
# the constraint are the same as the CP model version 2
# number of items > number of couriers
constraints.append(n > m)

# Sum of item sizes cannot be bigger than total load size
constraints.append(items_total <= Sum([load_size[i] for i in COURIERS]))

# each value in the path is between 0 and n
for c in COURIERS:
    for p in ITEMS_PATH:
        constraints.append(And(paths[c][p] >= 0, paths[c][p] <= n))

# each item should have size > 0
for i in ITEMS:
    constraints.append(item_size[i] > 0)

# each courier should have load size > 0
for c in COURIERS:
    constraints.append(load_size[c] > 0)

# Each item is delivered at most by one courier, only n+1 value can be repeated as it's used as a filler element
for i in ITEMS:
    constraints.append(Sum([If(paths[c][p] == i, 1, 0) for c in COURIERS for p in ITEMS_PATH]) == 1)

# Each courier delivers at least one item
for c in COURIERS:
    constraints.append(Sum([If(paths[c][p] != n, 1, 0) for p in ITEMS_PATH]) >= 1)

# First element and last element in path is the origin (n+1)
for c in COURIERS:
    constraints.append(paths[c][0] == n )
    constraints.append(paths[c][n] == n )

# Each courier's items sizes must not excede his load size
for i in ITEMS:
    for c in COURIERS:
        constraints.append(Sum([If(paths[c][p] == i, item_size[i], 0) for p in ITEMS_PATH]) <= load_size[c])
    
constraints

[True,
 True,
 And(paths_0_1 >= 0, paths_0_1 <= 6),
 And(paths_0_2 >= 0, paths_0_2 <= 6),
 And(paths_0_3 >= 0, paths_0_3 <= 6),
 And(paths_0_4 >= 0, paths_0_4 <= 6),
 And(paths_0_5 >= 0, paths_0_5 <= 6),
 And(paths_1_1 >= 0, paths_1_1 <= 6),
 And(paths_1_2 >= 0, paths_1_2 <= 6),
 And(paths_1_3 >= 0, paths_1_3 <= 6),
 And(paths_1_4 >= 0, paths_1_4 <= 6),
 And(paths_1_5 >= 0, paths_1_5 <= 6),
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 If(paths_0_1 == 0, 1, 0) +
 If(paths_0_2 == 0, 1, 0) +
 If(paths_0_3 == 0, 1, 0) +
 If(paths_0_4 == 0, 1, 0) +
 If(paths_0_5 == 0, 1, 0) +
 If(paths_1_1 == 0, 1, 0) +
 If(paths_1_2 == 0, 1, 0) +
 If(paths_1_3 == 0, 1, 0) +
 If(paths_1_4 == 0, 1, 0) +
 If(paths_1_5 == 0, 1, 0) ==
 1,
 If(paths_0_1 == 1, 1, 0) +
 If(paths_0_2 == 1, 1, 0) +
 If(paths_0_3 == 1, 1, 0) +
 If(paths_0_4 == 1, 1, 0) +
 If(paths_0_5 == 1, 1, 0) +
 If(paths_1_1 == 1, 1, 0) +
 If(paths_1_2 == 1, 1, 0) +
 If(paths_1_3 == 1, 1, 0) +
 If(paths_1_4 == 1, 1, 0) +
 If(paths_1_5

Solving the model:

In [86]:
for i in ITEMS:
    print(i)

0
1
2
3
4
5


In [90]:
# add all the constraints to the solver
s = Solver()
s.add(constraints)
# solve
if s.check() == sat:
    m = s.model()
    # print the model in a readable format
    paths_values = []
    for i in range(len(paths)):
        paths_values.append([m.evaluate(paths[i][j]) for j in range(len(paths[i]))])
    print(paths_values)
else:
    print("unsat")


[[6, 2, 4, 6, 3, 6, 6], [6, 0, 6, 6, 1, 5, 6]]
